# Run all cells in this notebook

Once they run, follow the instructions near the bottom for using the viewer/setting the image that is loaded

In [1]:
import numpy as np
from astrowidgets import ImageWidget
from photutils import centroid_com
import ipywidgets as ipw
from astropy.stats import sigma_clipped_stats
from astropy.table import Table
%matplotlib inline
import matplotlib.pyplot as plt
from notebook_functions import *

In [2]:
iw = ImageWidget(image_width=500, image_height=500, use_opencv=True)

In [3]:
iw.load_fits('/Users/mcraig/Documents/Research/ccd-reduction-photometry/notebooks/2018-07-23/kelt-16-S001-R001-C052-r.fit')
iw

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


ImageWidget(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\x…

In [4]:
np.seterr(all='ignore')

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [ ]:
def show_event(viewer, event, datax, datay):
    
    i = viewer._viewer.get_image()
    data = i.get_data()
    pad = 15
    x = int(np.floor(event.data_x))
    y = int(np.floor(event.data_y))
    cnt = 0
    sub_data = data[y - pad:y + pad, x - pad:x + pad] #- med
    _, sub_med, _ = sigma_clipped_stats(sub_data)
    #sub_med = 0
    foo, moo = centroid_com(sub_data - sub_med)
    cenx = foo + x - pad
    ceny = moo + y - pad
    cen = np.array([foo + x - pad, moo + y - pad])
    ceno = np.array([-100, -100])
    while cnt <= 10 and (np.abs(np.array([foo, moo]) - pad).max() >3 or np.abs(cen - ceno).max() > 0.1):
       # print(cnt, foo, moo)
        x = int(np.floor(foo)) + x - pad
        y = int(np.floor(moo)) + y - pad
        sub_data = data[y - pad:y + pad, x - pad:x + pad] #- med
        _, sub_med, _ = sigma_clipped_stats(sub_data)
        #sub_med = 0
        mask = (sub_data - sub_med) < 0
        foo, moo = centroid_com(sub_data - sub_med, mask=mask)
        ceno = cen
        cen = np.array([foo + x - pad, moo + y - pad])
#             print(cen)
#             print(cen - ceno)
        cnt += 1

    viewer.add_markers(Table(data=[[cen[0]], [cen[1]]], names=['x', 'y']))
    #print(foo, moo)
    yd, xd = np.indices((sub_data.shape))
    r = np.sqrt((xd - foo)**2 + (yd - moo)**2)
    r_exact = r.copy()
    r = r.astype(np.int)
    tbin = np.bincount(r.ravel(), sub_data.ravel())
    rbin = np.bincount(r.ravel(), r_exact.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    ravg = rbin / nr
    adjust_max = radialprofile.max() - sub_med
    scaled_profile = (radialprofile - sub_med) / adjust_max
    scaled_exact_counts = (sub_data - sub_med) / adjust_max
    out.clear_output(wait=True)
    with out:
       # print(dir(event))
        #print(event.data_x, event.data_y)
        plt.clf()
        #sub_med += med
        seeing_plot(r_exact, scaled_exact_counts, ravg, scaled_profile, 4.5,
                'Some Image Name', file_name='some_name', gap=6, annulus_width=13)
        plt.show()
    out2.clear_output(wait=True)
    with out2:
        tbin2 = np.bincount(r.ravel(), (sub_data - sub_med).ravel())
        counts = np.cumsum(tbin2)
        mag_diff = -2.5 * np.log10(counts/counts.max())
        plt.plot(range(len(radialprofile)), counts)
        plt.xlim(0, 20)
        #plt.ylim(0.2, 0)
        plt.grid()
        sub_blot = sub_data.copy()
        sub_blot[10:30, 10:30] = np.nan
        sub_std = np.nanstd(sub_blot)
        plt.title('Net counts in aperture std {:.2f} med {:.2f}'.format(sub_std, sub_med))
        sub_pois = (sub_data - sub_med)
        e_sky = np.sqrt(sub_med)
        rn = 10
        sub_noise_sq = np.sqrt(sub_pois ** 2 + sub_std ** 2) ** 2
        nbin = np.sqrt(np.bincount(r.ravel(), (sub_noise_sq).ravel()))
        plt.xlabel('Aperture radius')
        plt.show()
    out3.clear_output(wait=True)
    with out3:
        poisson = np.sqrt(np.cumsum(tbin2))
        error = np.sqrt(poisson ** 2 + np.cumsum(nr) * (e_sky ** 2 + rn ** 2))
        snr = np.cumsum(tbin2) / error
        snr_max = snr[:20].max()
        plt.plot(range(len(radialprofile)), snr)
        plt.title('Signal to noise ratio {}'.format(snr.max()))
        plt.xlim(0, 20)
        #plt.ylim(0, 2)
        plt.xlabel('Aperture radius')
        plt.grid()
        plt.show()

In [5]:
big_box = box(iw)
build(iw)
bind_map = iw._viewer.get_bindmap()

# Displays the event map...
#bind_map.eventmap


gvc = iw._viewer.get_canvas()
#gvc.add_callback('ms_left-my_custom_event-down', show_event)
#bind_map.map_event(None, ('shift',), 'ms_left', 'cursor')
gvc.add_callback('cursor-down', make_show_event(iw))

In [ ]:
bind_map.eventmap

## The cell below displays the viewer

### Left-click and drag to pan
### Scroll or use the +/- keys to zoom in/out
### Ctrl-left-click to display seeing profiles for stars

In [6]:
big_box

## Run cell below to load/change image

In [ ]:
iw.load_fits('/Volumes/My Passport for Mac/Strong 2019/all-18-7-23/2018-07-23-reduced/kelt-16-S001-R001-C052-r.fit')

In [ ]:
from astrowidgets import __version__
print (__version__)